In [1]:
from pyspark.sql import SparkSession
import pandas as pd

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

In [4]:
# Specify the path to your CSV file
csv_file_paths = []

# Read the CSV file into a DataFrame
df = spark.read.csv("salaries")

In [5]:
# Show the DataFrame
df.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|      _c0|             _c1|            _c2|                 _c3|   _c4|            _c5|          _c6|               _c7|         _c8|             _c9|        _c10|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
|     2023|              SE|             FT|Principal Data Sc...| 80000|            EUR|        85847|                ES|         100|              ES|           L|
|     2023|              MI|             CT|         ML Engineer| 30000|            USD|        30000|                US|         100|              US|           S|
|     2023